In [1]:
!pip install -r ../requirements.txt 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd

In [3]:
def extract_products_by_category(category):
    url = f"https://de.openfoodfacts.org/cgi/search.pl?action=process&tagtype_0=categories&tag_contains_0=contains&tag_0={category}&page_size=100&json=true"
    response = requests.get(url)
    
    if response.status_code == 200:
        products_data = response.json()["products"]
        products_df = pd.DataFrame(products_data)
        return products_df
    else:
        print(f"Failed to fetch data for {category}")
        return None

categories = ["Drinks", "Dairy Products", "Beverages", "Nuts", "Sauces", "Sandwiches", "Snacks", "Meat alternatives", "Ready meals"]

combined_products = pd.DataFrame()

# Extract and combine products for each category
for category in categories:
    products_df = extract_products_by_category(category)
    if products_df is not None:
        combined_products = pd.concat([combined_products, products_df], ignore_index=True)

# Remove duplicates
combined_products.drop_duplicates(subset="product_name", keep="first", inplace=True)

combined_file_name = "combined_products.csv"
combined_products.to_csv('../data/'+combined_file_name, index=False)
print(f"Combined products saved to {combined_file_name}")

Combined products saved to combined_products.csv
